In [1]:
import numpy as np
import matplotlib.pyplot as plt
from layers import *

In [2]:
arr1 = np.random.randn(2, 3)
arr2 = np.random.randn(2, 3)

print(arr1)
print(arr2)

[[ 0.50286098 -0.26170417 -1.1253311 ]
 [ 1.67317434 -1.20965068 -0.8729599 ]]
[[-0.58473428  1.73489488 -0.84686156]
 [ 0.18784678  1.64413975  1.39854022]]


In [3]:
np.abs(arr1)

array([[ 0.50286098,  0.26170417,  1.1253311 ],
       [ 1.67317434,  1.20965068,  0.8729599 ]])

In [5]:
np.max(arr1)


1.6731743389742018